In [1]:
import pandas as pd, numpy as np
import re, glob

In [2]:
eu = [
    "Austria", "Belgium", "Bulgaria","Croatia","Cyprus","Czech Republic","Denmark","Estonia","Finland","France","Germany",
    "Greece","Hungary","Ireland","Italy","Latvia","Lithuania","Luxembourg","Malta",
    "Netherlands","Poland","Portugal","Romania","Slovakia","Slovenia","Spain","Sweden",'United Kingdom']

In [3]:
master = pd.DataFrame()
codes = []
for x in glob.glob('per_country_full/*.csv'):
    country = x.split('/')[1].split('_')[0]
    if country in eu:
        print(country)
        df = pd.read_csv(x)
        print(df.shape[0])
        df = df.drop(columns=['Unnamed: 0'])
        master = pd.concat([master,df])
    

Austria
45387
Sweden
17387
Estonia
16813
Slovenia
40029
Croatia
31161
Italy
22702
Cyprus
12655
Portugal
48184
Netherlands
132586
Romania
11339
Greece
34763
Germany
88750
Ireland
30425
Belgium
155330
Czech Republic
26224
Finland
22643
France
9421
Denmark
19049
Hungary
47666
Poland
29081
Bulgaria
12369
Slovakia
30484
Spain
149575
Latvia
2031
Luxembourg
30845
Lithuania
44980
United Kingdom
18158


In [4]:
master.shape

(1130037, 15)

In [5]:
master['year']=master['manifesto_id'].apply(lambda x: pd.to_datetime(x.split("_")[1][:4]).year)
master['party_id']=master['manifesto_id'].apply(lambda x: x.split("_")[0])

In [6]:
# DROP ALL BEFORE 1990
master = master[master['year']>=1990]
master.shape

(1115438, 17)

In [7]:
master.reset_index(inplace=True)

In [8]:
# manifestos per language
master['language'].value_counts()

language
dutch         201807
german        156427
spanish       120404
french         90682
english        49287
portuguese     48149
hungarian      47666
greek          47418
lithuanian     44980
slovenian      40029
croatian       31161
slovak         30261
polish         28550
czech          26224
catalan        23182
finnish        22303
italian        21836
danish         18934
swedish        17624
estonian       16813
bulgarian      12365
romanian       11339
galician        5966
latvian         2031
Name: count, dtype: int64

In [9]:
# manifestos per country
master.groupby(['country'])['manifesto_id'].nunique()

country
Austria            50
Belgium            79
Bulgaria           25
Croatia            54
Cyprus             20
Czech Republic     31
Denmark            79
Estonia            24
Finland            62
France             38
Germany            50
Greece             49
Hungary            28
Ireland            41
Italy              69
Latvia             30
Lithuania          42
Luxembourg         17
Netherlands        96
Poland             43
Portugal           41
Romania            29
Slovakia           46
Slovenia           41
Spain             104
Sweden             60
United Kingdom     22
Name: manifesto_id, dtype: int64

In [10]:
# DROP ALL THAT HAVEN'T BEEN ANNOTATED AT QS LEVEL, WE WILL COME BACK TO THESE IN LATER SCRIPTS
mvc = master['manifesto_id'].value_counts()
master = master[~master['manifesto_id'].isin(mvc[mvc==1].index)]
master.reset_index(inplace=True)

In [11]:
master.shape

(1115148, 19)

In [13]:
newm = pd.DataFrame()
for i,r in enumerate(master['manifesto_id'].unique()):
    s = master[master['manifesto_id']==r]
    qs = [r+'_'+str(0)*(4-len(str(x)))+str(x) for x in range(s.shape[0])]
    s['qs_new']=qs
    newm = pd.concat([newm,s])
    # for q in qs:
    #     qs_ids.append(q)

/var/folders/m2/v5b9_lws6kj40bh4vjzm0x980000gn/T/ipykernel_91332/1243778517.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s['qs_new']=qs
/var/folders/m2/v5b9_lws6kj40bh4vjzm0x980000gn/T/ipykernel_91332/1243778517.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s['qs_new']=qs
/var/folders/m2/v5b9_lws6kj40bh4vjzm0x980000gn/T/ipykernel_91332/1243778517.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [14]:
newm.reset_index(inplace=True,drop=True)

In [15]:
newm.shape, master.shape

((1115148, 20), (1115148, 19))

In [ ]:
newm.to_csv('all_eu_raw_mpd_27Apr.csv',header=True,index=False)